### *Loading the fine-tuned model*

### *Installing the requirements*

In [ ]:
! pip install -qqq datasets rouge-score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
import bert_score
from datasets import Dataset
from rouge_score import rouge_scorer

### *Loading the dataset*

In [ ]:
#@title Function to calculate ROUGE scores

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge_scores(reference, hypothesis):
    scores = scorer.score(reference, hypothesis)
    return {
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_precision': scores['rouge2'].precision,
        'rouge2_recall': scores['rouge2'].recall,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall,
        'rougeL_f1': scores['rougeL'].fmeasure
    }

In [ ]:
test_df = pd.read_csv("Model_output.csv")
test_df_dataset = Dataset.from_pandas(test_df)

In [ ]:
from tqdm import tqdm

predictions = df["model_predictions"].tolist()

# Calculate ROUGE scores
results = []

for i, (row, prediction) in tqdm(enumerate(zip(test_df_dataset, predictions)), total=len(predictions), desc="Calculating ROUGE scores"):
    summary = row["Perspective-based Summary"]
    rouge_scores = calculate_rouge_scores(summary, prediction)
    results.append(rouge_scores)

In [ ]:
# Convert the results into a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Merge the actual dataset and the adjusted scores dataset on the index
merged_df = pd.merge(test_df, results_df, left_index=True, right_index=True)

### *BERT Score*

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
summary = merged_df["Perspective-based Summary"].tolist() # actual summary

# Compute BERTScore
precision, recall, f1 = bert_score.score(summary, predictions, lang='en', verbose=True, device=device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 5.07 seconds, 19.74 sentences/sec


In [ ]:
# Add BERTScore to the DataFrame
merged_df['BERTScore_Precision'] = precision.numpy()
merged_df['BERTScore_Recall'] = recall.numpy()
merged_df['BERTScore_F1'] = f1.numpy()

### *Saving the scores*

In [ ]:
# Save the result to a new CSV file
merged_df.to_csv('Evaluation_scores.csv', index=False)

### *Overall Average Scores*

In [ ]:
df = pd.read_csv('Evaluation_scores.csv')

In [ ]:
# Calculate aggregated results
aggregated_results = {
    'rouge1_precision': df['rouge1_precision'].mean(),
    'rouge1_recall': df['rouge1_recall'].mean(),
    'rouge1_f1': df['rouge1_f1'].mean(),
    'rouge2_precision': df['rouge2_precision'].mean(),
    'rouge2_recall': df['rouge2_recall'].mean(),
    'rouge2_f1': df['rouge2_f1'].mean(),
    'rougeL_precision': df['rougeL_precision'].mean(),
    'rougeL_recall': df['rougeL_recall'].mean(),
    'rougeL_f1': df['rougeL_f1'].mean(),
    'BERTScore_Precision': df['BERTScore_Precision'].mean(),
    'BERTScore_Recall': df['BERTScore_Recall'].mean(),
    'BERTScore_F1': df['BERTScore_F1'].mean()
}
aggregated_rouge_scores = pd.DataFrame(aggregated_results, index=['Value'])
aggregated_rouge_scores